# **Part III - Transactions**

In [6]:
import requests
from pprint import pprint
import pandas as pd
import os
import dask 
from datetime import datetime, timedelta


# **Get data from API**

TO DO : 

- Run la cell en dessous 
- Avant, il faut aller [ici](https://docs.coinapi.io/), rentrer son adresse mail et clicker sur `get a free api key`
- Copier coller la clé dans un fichier dans le directory du notebook : nom du ficher --> `path_file_API`
- Aller dans le fichier `periods_API.txt`
- Choisir l'index (0,7,14,...) --> dans `index`

- Joao : Tu peux faire de 30 à 35 (des fois ça bug, donc va dans periods_API.txt, supprime les colonnes déjà Ok et retente avec une autre clé api).

In [50]:
def divide_time_periods() : 
    # Define the start and end dates
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2021, 9, 1)

    # Generate a list of dates
    date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    datetime_strings = [dt.strftime('%Y-%m-%d') for dt in date_list]
    
    #Divide into slots of 7 (we can only make 10 calls per day)
    slot_size = 7
    date_slots = [datetime_strings[i:i+slot_size] for i in range(0, len(datetime_strings), slot_size)]
    
    with open("data/periods_API.txt", "w") as file : 
        for i in range(0, len(datetime_strings), slot_size) :
            data = str(i) + " : " + " ".join(datetime_strings[i:i+slot_size])
            file.write(data + "\n")

In [138]:
# Retrieve API key
def retrieve_api_key(name_file) : 
    api_key = None
    with open(name_file) as file:
        api_key = file.readline().strip()
    return api_key


# Retrieve historical data on Trades
def get_historical_data(type, headers, index, day, save_dir) :
    
    parameters = {  
        "symbol_id": "BITSTAMP_SPOT_BTC_USD",
        "time_start" : day+'T00:00',
        "time_end" : day+'T23:59',
        "limit" : 100000, 
    }
    r = requests.get(f"https://rest.coinapi.io/v1/{type}/BITSTAMP_SPOT_BTC_USD/history", 
                     headers=headers, params=parameters)

    # Check if the request was successful (status code 200)
    if r.status_code == 200:
        # Parse and print the trade data
        data = r.json()
        df = pd.DataFrame(data)
        
        #Save as csv 
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        name_file = str(index) + "_" + day
        df.to_parquet(save_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")

    else:
        # Print an error message if the request was unsuccessful
        print(f"Error: {r.status_code} - {r.text}")

def main_get_historical_data(type, path_file_API, path_file_periods, index) : 
    api_key = retrieve_api_key(path_file_API)
    headers = {"X-CoinAPI-Key": api_key}

    days_list = None
    #Open the file and get the list of index to scrap (based on index)
    with open(path_file_periods) as file : 
        lines = file.readlines()
        for line in lines : 
            if line.split(":")[0].strip() == str(index) :
                days_list = line.split(":")[1][1:-1].split(" ")
    
    for day in days_list : 
        get_historical_data(type, headers, index, day, save_dir = "data/raw/trades/")


#To call every day 
def main_get_historical_trades(path_file_API, path_file_periods, index) : 
    main_get_historical_data("trades", path_file_API, path_file_periods, index)

def main_get_historical_quotes(path_file_API, path_file_periods, index) : 
    main_get_historical_data("quotes", path_file_API, path_file_periods, index)


#main_get_historical_trades("API_key_affolter.txt", "data/periods_API.txt", 175)
#main_get_historical_trades("API_key_ben.txt", "data/periods_API.txt", 182)
#main_get_historical_trades("API_key_epfl.txt", "data/periods_API.txt", 189)
#main_get_historical_trades("API_key_joanne.txt", "data/periods_API.txt", 196)
main_get_historical_trades("API_key_reve.txt", "data/periods_API.txt", 203)
main_get_historical_trades("API_key_data.txt", "data/periods_API.txt", 210)
main_get_historical_trades("API_key_benProut.txt", "data/periods_API.txt", 217)
main_get_historical_trades("API_key_anne.txt", "data/periods_API.txt", 224)
main_get_historical_trades("API_key_benjiplayer.txt", "data/periods_API.txt", 231)
main_get_historical_trades("API_key_toto.txt", "data/periods_API.txt", 238)
#main_get_historical_trades("API_key_benj.txt", "data/periods_API.txt", 133)
#main_get_historical_trades("API_key_spoteamfy.txt", "data/periods_API.txt", 140)
#main_get_historical_trades("API_key_toto_reve.txt", "data/periods_API.txt", 140)
#main_get_historical_trades("API_key_bene.txt", "data/periods_API.txt", 147)
#main_get_historical_trades("API_key_anne_reve.txt", "data/periods_API.txt", 154)
#main_get_historical_trades("API_key_bene3.txt", "data/periods_API.txt", 168)

In [110]:
#Check which files don't have all data collected 

import glob 

allfiles = glob.glob("data/raw/trades/*")

not_all_transactions = []
for file in allfiles : 
    df = pd.read_parquet(file)
    if len(df) == 100000 :
        name_file = file.split("\\")[1]
        not_all_transactions.append(name_file)

not_all_transactions

['28_2021-01-29', '49_2021-02-23', '7_2021-01-08', '7_2021-01-11']